In [ ]:
import pandas as pd
import numpy as np

from turtle import color
import plotly.graph_objects as go
import plotly.express as px

import matplotlib.pyplot as plt
import matplotlib_inline
import statsmodels as sm


from sklearn.model_selection import train_test_split as skl

from sklearn.ensemble import BaggingRegressor
#### Bagging con ___ como estimador base
from sklearn import neighbors
from sklearn.ensemble import RandomForestRegressor
import sklearn
# CONTEXTO EN CLASIFICACIÓN
sklearn.ensemble.RandomForestRegressor
sklearn.ensemble.RandomForestClassifier
# importar librerías
from sklearn import tree

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn import metrics

In [ ]:
kernel='C:/Users/miavila/Desktop/PREDICIONES/'

df=pd.read_excel(kernel + 'AGRU_HOUR.xlsx')
df=pd.DataFrame(df,columns=['Operación_N','sumRecibidas','FechaT', # Fecha para concatenar con el set de pronostico de incidentes en el otro scrip
       'festivo', 'festividades','Dia_SemN', 'Dia_MesN','MesN','Hora24'])
       


In [ ]:
kernel2='C:/Users/miavila/Documents/U/PROYECTO P/variables exogenas/' # PARA X_NEW
X_new_train=pd.read_excel(kernel2 + 'X_new_hour.xlsx')
X_new_train = X_new_train.set_index('FechaT')
X_new_train.drop(columns=['Unnamed: 0','Fecha'],inplace=True)
X_new_train.drop(columns = ['MesN'],inplace=True)
fin_train = '2022-09-01 00:00:00'
fin_validacion = '2023-09-29 23:00:00'


In [ ]:
def iterar_por_nombres(df):
    # obtener los nombres únicos de la columna Operación_N
    nombres_unicos = df['Operación_N'].unique()
    
    # crear una lista vacía para almacenar los resultados
    resultados = []
    
    # iterar por cada nombre único
    for nombre in nombres_unicos:
            # filtrar el DataFrame para obtener los registros con el nombre específico
########CAMBIAR MESES PARA ENTRENAR
        df=df[df['MesN'].isin([1,2,3,4,5,6,7,8])]
        df_nombre = df[df['Operación_N'] == nombre]
        
        # convertir la columna FechaT en el índice
        df_nombre=df_nombre.sort_values(by='FechaT') #ascending=False 
        df_nombre.set_index('FechaT', inplace=True)
    
        #modelo
        df_nombre=pd.DataFrame(df_nombre,columns=[ 'sumRecibidas', # no se cuenta la columna con el nombre de la operación
        'festivo', 'festividades','Dia_SemN', 'Dia_MesN','MesN','Hora24'])
        #cargar nuevo X_TRAIN (set de predictores para año futuro)
        datos_train = df_nombre.drop(columns = ['MesN'])
        #datos_test = X_new_train.loc[fin_train:fin_validacion, :]
        #print(f"Fechas train      : {datos_train.index.min()} --- {datos_train.index.max()}  (n={len(datos_train)})")
        #datos_test=datos_test.drop(['2022-07-31 23:00:00'])
        #print(f"Fechas test       : {datos_test.index.min()} --- {datos_test.index.max()}  (n={len(datos_test)})")
        ###################################################################################################################
        X_train, y_train= datos_train.drop(columns = ['sumRecibidas']),datos_train.sumRecibidas 
        # Fitting Random Forest Regression to the dataset
        # import the regressor
        from sklearn.ensemble import RandomForestRegressor
        # create regressor object
        regressor = RandomForestRegressor(n_estimators =50,max_features=5,max_depth=25,min_samples_split=0.0001, random_state = 123456)
        # fit the regressor with x and y data
        modelRF=regressor.fit(X_train, y_train) 
        modelRF
        tree_pred =modelRF.predict(X_new_train)
        y_predd = pd.DataFrame(np.round(tree_pred))
        y_predd.index=X_new_train.index
        y_predd.rename(columns={0:'Predicción'},inplace=True)
        y_predd
        y_predd['Operación']='{oper}'.format(oper=nombre)
        # guardar los resultados en la lista
        resultados.append(y_predd)
           
         # concatenar los resultados horizontalmente 1 | vertical 0
        df_resultado = pd.concat(resultados, axis=0)
        
    
        df_resultado = df_resultado[df_resultado['Operación'] != 'CAV']
        pass
        #para REDENCIÓN SOAT SOLO ENTRENADO DE JULIO A AGOSTO_ POR QUE NO HAY LLAMADAS DESDE ENERO HASTA MAYO DE ESTA OPERACIÓN
########CAMBIAR MESES PARA ENTRENAR SOLO CAV
        df=df[df['MesN'].isin([6,7,8])]
        CAV= df[df['Operación_N'] == 'CAV']
        
        datos_train_CAV=CAV.loc['2022-06-01 00:00:00':'2022-09-29 23:00:00'] #%%%%%%%%%%%%%%%%%


        df_nombre_CAV=pd.DataFrame(CAV,columns=[ 'sumRecibidas', # no se cuenta la columna con el nombre de la operación
        'festivo', 'festividades','Dia_SemN', 'Dia_MesN','MesN','Hora24'])
        
        datos_train_CAV = df_nombre_CAV.drop(columns = ['MesN'])
        

        X_train_CAV, y_train_CAV= datos_train_CAV.drop(columns = ['sumRecibidas']),datos_train_CAV.sumRecibidas 
        # Fitting Random Forest Regression to the dataset
        # import the regressor
        from sklearn.ensemble import RandomForestRegressor
        # create regressor object
        regressor = RandomForestRegressor(n_estimators =50,max_features=5,max_depth=25,min_samples_split=0.0001, random_state = 123456)
        # fit the regressor with x and y data
        modelRF=regressor.fit(X_train_CAV, y_train_CAV) 
        modelRF
        tree_pred2 =modelRF.predict(X_new_train)
        y_predd_CAV = pd.DataFrame(np.round(tree_pred2))
        y_predd_CAV.index=X_new_train.index
        y_predd_CAV.rename(columns={0:'Predicción'},inplace=True)
        y_predd_CAV
        y_predd_CAV['Operación']='{oper}'.format(oper='CAV')
        # guardar los resultados en la lista
        

           # concatenar los resultados horizontalmente 1 | vertical 0
        df_resultado2 = pd.concat([df_resultado,y_predd_CAV], axis=0)







        # guardar el resultado en un archivo excel   
        #df_resultado.to_excel('predicciones_Mdd.xlsx')
             
    return df_resultado2
pass
base=iterar_por_nombres(df)
base.to_excel('PRUEBACCCC.xlsx')
base

Predicción  Operación
FechaT                                    
2022-09-01 00:00:00         0.0  Biometria
2022-09-01 01:00:00         0.0  Biometria
2022-09-01 02:00:00         0.0  Biometria
2022-09-01 03:00:00         0.0  Biometria
2022-09-01 04:00:00         0.0  Biometria
...                         ...        ...
2023-09-29 19:00:00         0.0        CAV
2023-09-29 20:00:00         1.0        CAV
2023-09-29 21:00:00         0.0        CAV
2023-09-29 22:00:00         0.0        CAV
2023-09-29 23:00:00         0.0        CAV

[104016 rows x 2 columns]

In [ ]:
CAV=base[base['Operación']=='CAV']

In [ ]:
df=base.copy()

In [ ]:
BIO=df[df['Operación']=='Biometria']
CON=df[df['Operación']=='Conservacion']
EP=df[df['Operación']=='Ejecutivo Premium']
EB=df[df['Operación']=='Especializados Bloqueos']
EBF=df[df['Operación']=='Especializados Bloqueos F']
ME=df[df['Operación']=='Moneda Extranjera - Home']
OR=df[df['Operación']=='Orquestador']
PJH=df[df['Operación']=='Persona Juridica - Home']
CAV=df[df['Operación']=='CAV']
VIP=df[df['Operación']=='VIP PJ']

In [ ]:
from turtle import color
import plotly.graph_objects as go
import plotly.express as px
# Create figure
    # ORDENA POR FECHA
        #P=P.sort_values(by='FechaT') # PENSILVANIA
        #P.index = range(P.shape[0])
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(CAV.index), y=list(CAV.Predicción)))
    #px.line(x=list(UPN.FechaT), y=list(UPN.sumRecibidas)))

# Set title
fig.update_layout(
    title_text="SERIE DE TIEMPO "
)

# Add range slider
fig.update_layout(autosize=True,width=1300,
    height=700,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1 DIA",
                     step="day",
                     stepmode="backward"),
                      dict(count=7,
                     label="7 DIAS",
                     step="day",
                     stepmode="backward"),
                      dict(count=15,
                     label="15 DIAS",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1 MES",
                     step="month",
                     stepmode="backward"),
                     dict(count=2,
                     label="2 MESES",
                     step="month",
                     stepmode="backward"),
                     dict(count=3,
                     label="3 MESES",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6 MESES",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="ALL",
                     step="year",
                     stepmode="todate"),
                
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)
fig.add_vrect(x0="2023-01-6", x1="2023-01-28 23:00:00", 
              annotation_text="TEMPORADA VACACIONAL", annotation_position="top left",
              fillcolor="red", opacity=0.25, line_width=3)
fig.add_vrect(x0="2023-04-01", x1="2023-04-09 23:00:00", 
              annotation_text="SEMANA SANTA", annotation_position="top left",
              fillcolor="red", opacity=0.25, line_width=3)
 
fig.show()